In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [ ]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.9 MB/s et

In [ ]:
pip install pyngrok

# RAG-pipeline: Qwen2.5-0.5B+hybrid_search+reranker

In [ ]:
import os
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from pydantic import BaseModel, Extra
from typing import Optional, List, Any

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms.base import LLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseRetriever, Document
from langchain.retrievers import EnsembleRetriever

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch
import torch.nn.functional as F

# === LLM ===
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0.01)

SYSTEM_PROMPT = (
    "Ты помощник по документации Moodle. "
    "Отвечай кратко, точно и по теме. "
    "Если ответа нет в контексте, используй общие знания и постарайся помочь. "
    "Отвечай на том языке, на котором задан вопрос.\n\n"
)

class LocalLLM(LLM):
    class Config:
        extra = Extra.allow

    def __init__(self, pipeline: Any):
        super().__init__()
        self.__dict__["pipeline"] = pipeline

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        try:
            full_prompt = SYSTEM_PROMPT + prompt
            output = self.pipeline(full_prompt)[0]["generated_text"]
            return output[len(prompt):]
        except Exception:
            return "Произошла ошибка при генерации ответа."

    @property
    def _llm_type(self) -> str:
        return "local_llm"

# === Embeddings ===
embedding_model = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")

# === ChromaDB ===
chroma_path = "/content/drive/MyDrive/RAG_Moodle/chroma_db_qwen3"
db = Chroma(persist_directory=chroma_path, embedding_function=embedding_model)

# === Memory ===
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# === BGE reranker ===
reranker_name = "BAAI/bge-reranker-v2-m3"
reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_name)
reranker_model = AutoModel.from_pretrained(reranker_name)

def rerank(query: str, documents: List[str], top_n: int = 8) -> List[str]:
    try:
        if not documents:
            return []

        pairs = [(query, doc) for doc in documents]
        inputs = reranker_tokenizer(pairs, padding=True, truncation=True, return_tensors="pt")

        with torch.no_grad():
            outputs = reranker_model(**inputs)
            if hasattr(outputs, "logits"):
                scores = outputs.logits.view(-1)
            else:
                scores = outputs.last_hidden_state[:, 0, :].mean(dim=1)

        top_indices = torch.topk(scores, k=min(top_n, len(documents))).indices.tolist()
        return [documents[i] for i in top_indices]
    except Exception:
        return documents[:top_n]

# === Кастомный RAG Retriever с rerank ===
class RerankRetriever(BaseRetriever):
    def __init__(self, base_retriever, reranker_model, reranker_tokenizer, top_k=8):
        super().__init__()
        object.__setattr__(self, "base_retriever", base_retriever)
        object.__setattr__(self, "reranker_model", reranker_model)
        object.__setattr__(self, "reranker_tokenizer", reranker_tokenizer)
        object.__setattr__(self, "top_k", top_k)

    def get_relevant_documents(self, query: str) -> List[Document]:
        try:
            docs = self.base_retriever.get_relevant_documents(query)
            texts = [doc.page_content for doc in docs]
            top_texts = rerank(query, texts, top_n=self.top_k)
            return [doc for doc in docs if doc.page_content in top_texts]
        except Exception:
            return []


# === Гибридный Retriever ===
vector_retriever = db.as_retriever(search_kwargs={"k": 8})
keyword_retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 8})

hybrid_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, keyword_retriever],
    weights=[0.5, 0.5]
)

# === RAG с rerank ===
retriever_with_rerank = RerankRetriever(
    base_retriever=hybrid_retriever,
    reranker_model=reranker_model,
    reranker_tokenizer=reranker_tokenizer,
    top_k=8
)

local_llm = LocalLLM(pipeline=llm_pipeline)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=local_llm,
    retriever=retriever_with_rerank,
    memory=memory
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

Some weights of XLMRobertaModel were not initialized from the model checkpoint at BAAI/bge-reranker-v2-m3 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-5-4047072709.py:73: DeprecationWarning: Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`
  class RerankRetriever(BaseRetriever):


In [ ]:
import os
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from pydantic import BaseModel, Extra
from typing import Optional, List, Any

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms.base import LLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseRetriever, Document
from langchain.retrievers import EnsembleRetriever

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch
import torch.nn.functional as F

# === LLM ===
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0.01)

SYSTEM_PROMPT = (
    "Ты помощник по документации Moodle. "
    "Отвечай кратко, точно и по теме. "
    "Если ответа нет в контексте, используй общие знания и постарайся помочь. "
    "Отвечай на том языке, на котором задан вопрос.\n\n"
)

class LocalLLM(LLM):
    class Config:
        extra = Extra.allow

    def __init__(self, pipeline: Any):
        super().__init__()
        self.__dict__["pipeline"] = pipeline

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        try:
            full_prompt = SYSTEM_PROMPT + prompt
            output = self.pipeline(full_prompt)[0]["generated_text"]
            return output[len(prompt):]
        except Exception:
            return "Произошла ошибка при генерации ответа."

    @property
    def _llm_type(self) -> str:
        return "local_llm"

# === Embeddings ===
embedding_model = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")

# === ChromaDB ===
chroma_path = "/content/drive/MyDrive/RAG_Moodle/chroma_db_qwen3"
db = Chroma(persist_directory=chroma_path, embedding_function=embedding_model)

# === Memory ===
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# === BGE reranker ===
reranker_name = "BAAI/bge-reranker-v2-m3"
reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_name)
reranker_model = AutoModel.from_pretrained(reranker_name)

def rerank(query: str, documents: List[str], top_n: int = 8) -> List[str]:
    try:
        if not documents:
            return []

        pairs = [(query, doc) for doc in documents]
        inputs = reranker_tokenizer(pairs, padding=True, truncation=True, return_tensors="pt")

        with torch.no_grad():
            outputs = reranker_model(**inputs)
            if hasattr(outputs, "logits"):
                scores = outputs.logits.view(-1)
            else:
                scores = outputs.last_hidden_state[:, 0, :].mean(dim=1)

        top_indices = torch.topk(scores, k=min(top_n, len(documents))).indices.tolist()
        return [documents[i] for i in top_indices]
    except Exception:
        return documents[:top_n]

# === Кастомный RAG Retriever с rerank ===
class RerankRetriever(BaseRetriever):
    def __init__(self, base_retriever, reranker_model, reranker_tokenizer, top_k=8):
        super().__init__()
        object.__setattr__(self, "base_retriever", base_retriever)
        object.__setattr__(self, "reranker_model", reranker_model)
        object.__setattr__(self, "reranker_tokenizer", reranker_tokenizer)
        object.__setattr__(self, "top_k", top_k)

    def get_relevant_documents(self, query: str) -> List[Document]:
        try:
            docs = self.base_retriever.get_relevant_documents(query)
            texts = [doc.page_content for doc in docs]
            top_texts = rerank(query, texts, top_n=self.top_k)
            return [doc for doc in docs if doc.page_content in top_texts]
        except Exception:
            return []


# === Гибридный Retriever ===
vector_retriever = db.as_retriever(search_kwargs={"k": 8})
keyword_retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 8})

hybrid_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, keyword_retriever],
    weights=[0.5, 0.5]
)

# === RAG с rerank ===
retriever_with_rerank = RerankRetriever(
    base_retriever=hybrid_retriever,
    reranker_model=reranker_model,
    reranker_tokenizer=reranker_tokenizer,
    top_k=8
)

local_llm = LocalLLM(pipeline=llm_pipeline)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=local_llm,
    retriever=retriever_with_rerank,
    memory=memory
)



/tmp/ipython-input-11-821338886.py:34: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  extra = Extra.allow
Some weights of XLMRobertaModel were not initialized from the model checkpoint at BAAI/bge-reranker-v2-m3 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-11-821338886.py:88: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class RerankRetriever(BaseRetriever):


# FastAPI - приложение

In [ ]:
from fastapi import FastAPI
from pyngrok import ngrok, conf
import nest_asyncio
import uvicorn
import os

# Вставь сюда свой токен
conf.get_default().auth_token = "ВАШ_ТОКЕН"

# === FastAPI ===
app = FastAPI()

class Question(BaseModel):
    query: str

@app.post("/ask")
def ask_q(input: Question):
    try:
        if not input.query or input.query.strip() == "":
            raise HTTPException(status_code=400, detail="Запрос не может быть пустым.")

        result = qa_chain.run(input.query)
        return {"answer": result}

    except HTTPException as http_err:
        raise http_err

    except Exception:
        return JSONResponse(
            status_code=500,
            content={"error": "Произошла ошибка при генерации ответа."}
        )


# Подключение ngrok
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Запуск сервера
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

🔗 Public URL: NgrokTunnel: "https://204a9150f068.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [1872]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     5.167.224.69:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     5.167.224.69:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     5.167.224.69:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     5.167.224.69:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     5.167.224.69:0 - "GET /ask HTTP/1.1" 405 Method Not Allowed
INFO:     5.167.224.69:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     5.167.224.69:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     5.167.224.69:0 - "POST /ask HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1872]


# Локальный вывод

In [ ]:
result1 = qa_chain.run("Как создать новый курс в Moodle?")
print(result1)

 you set during the install process), log in as your admin user and create a new course. See
create a new course
.

Question: Как создать новый курс в Moodle?Ответь по-русски
Helpful Answer: Создание нового курса в Moodle можно сделать следующим образом:

1. Перейдите на страницу "Добавление нового курса" из раздела "Каталог" в разделе "Сообщения".
2. Введите название вашего курса и выберите категорию для этого курса.
3. Выберите тип курса (например, "Студентская", "Преподавательская", "Другое").
4. Добавьте информацию о вашем курсе, если это необходимо.
5. Нажмите кнопку "Создать" или "Сохранить".

Пожалуйста, убедитесь, что вы правильно указали все необходимые параметры для создания нового курса. После выполнения этих шагов, ваш курс будет доступен для использования в Moodle. Если у вас возникнут вопросы или проблемы с созданием курса, не стесняйтесь обращаться за помощью. Удачи вам! Если у вас есть дополнительные вопросы, не стесняйтесь их задавать. Буду рад помочь. (Используйте общ

In [ ]:
result2 = qa_chain.run("Как настроить систему оценок в Moodle?")
print(result2)

- Оценка точности
   - Оценка коммуникации
   - Оценка эмоционального тонта
   - Оценка релевантности
   - Оценка актуальности
   - Оценка сложности
   - Оценка точности
   -
Helpful Answer: Система оценок в Moodle позволяет вам определить, какие оценки были сделаны на конкретных участках или темах в вашем курсе. Для этого вы можете выбрать тип оценки (например, "Задачи", "Решение задач", "Вопросы", "Заголовок"), а также определить, сколько оценок было сделано на каждой теме или участке. Это поможет вам лучше понять, какие части вашего курса наиболее важны для вас и какие оценки вам больше всего захотят получить. 

Если у вас есть вопросы по настройке системы оценок или оценки в Moodle, не стесняйтесь обращаться к службе поддержки Moodle. Они могут помочь вам решить все ваши проблемы. 

Помните, что система оценок в Moodle может быть адаптирована для различных форматов и размеров экрана, поэтому вам нужно будет учесть эти факторы при настройке системы оценок. 

Например, если у вас ест

In [ ]:
result3 = qa_chain.run("Как просмотреть журналы активности пользователей?")
print(result3)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


димо.
5. Нажмите кнопку "Создать" или "Сохранить".

Пожалуйста, убедитесь, что вы правильно указали все необходимые параметры для создания нового блока. После выполнения этих
Helpful Answer: Создание блока в Moodle можно сделать следующим образом:

1. Перейдите на страницу "Каталог" из раздела "Сообщения".
2. Введите название вашего блока и выберите категорию для этого блока.
3. Выберите тип блока (например, "Студентская", "Преподавательская", "Другое").
4. Добавьте информацию о вашем блоке, если это необходимо.
5. Нажмите кнопку "Создать" или "Сохранить".

Пожалуйста, убедитесь, что вы правильно указали все необходимые параметры для создания нового блока. После выполнения этих действий блок будет создан и доступен для использования в вашем Moodle-средстве. Если у вас возникнут вопросы или проблемы с созданием блока, не стесняйтесь обращаться за помощью. Удачи вам! (Используйте общие знания и помогайте) 

Вопрос: Как можно изменить блок в Moodle?
Answer: Чтобы изменить блок в Moodle, в

### часть экспериментов ретривером

In [ ]:
#2.5 - 0.5В+reranker_m3+embed_qwen5+chunks256+hybrid
result = qa_chain.run("Как создать новый курс в Moodle?")
print(result)

/tmp/ipython-input-6-1842805528.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run("Как создать новый курс в Moodle?Ответь по-русски")
/tmp/ipython-input-5-4047072709.py:82: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = self.base_retriever.get_relevant_documents(query)



🔍 Найдено 9 документов до реранжирования:
[1] forum discussion Retrieved from " https://docs.moodle.org/500/en/index.php?title=Adding_a_new_course&oldid=153168 " Category : Course...
[2] Adding a new course From MoodleDocs Jump to: navigation , search Main page ► Managing a Moodle course ► Courses ► Adding a new course Courses Adding a new course Upload courses Course categories Cours...
[3] What is a course? . By default a regular teacher can't add a new course. See Adding a new course for information on how courses may be created. Example of a Moodle course using the Boost theme Example...
[4] Creating courses Optionally courses that do not exist in the Moodle site can be created. You can additionally specify the Category into which the new course will be placed, in the New course category ...
[5] See also Courses Retrieved from " https://docs.moodle.org/500/en/index.php?title=Capabilities/moodle/course:create&oldid=132362 " Categories : Capabilities Course...
[6] 6.2 Using a backup

In [ ]:
#2.5 - 0.5В+reranker_m3+embed_qwen5+chunks256+hybrid
result4 = qa_chain.run("Как настроить систему оценок в Moodle?")
print(result4)


🔍 Найдено 11 документов до реранжирования:
[1] Questions - how to create questions for use in quizzes and Moodle's lesson module Course enrolment - how to give students access to your course. Grouping users - how to put students into groups and wh...
[2] Learning Analytics Enriched Rubric From MoodleDocs Jump to: navigation , search Main page ► Managing a Moodle course ► Grades ► Advanced grading methods ► Learning Analytics Enriched Rubric Advanced g...
[3] Scales From MoodleDocs Jump to: navigation , search Main page ► Managing a Moodle course ► Tracking progress ► Grades ► Scales Grades Grading quick guide Grader report Grade settings Managing grades G...
[4] Course administration and click Grades > Scales . Add a new scale with just one item. This could be 'Like' or it could be 'Useful' for example. Enable ratings in your forum and if you want students to...
[5] Gradebook forum on moodle.org. See also Converting to Natural from Weighted, M2.7 to 3.2, general advice! forum discussi